In [1]:
"""
clone and install PySDM dependencies in Colab
"""
import os, sys
if 'google.colab' in sys.modules:
    %cd /content
    if not os.path.isdir('PySDM'):
        !git clone --depth 1 https://github.com/atmos-cloud-sim-uj/PySDM.git
        %cd PySDM
    else:
        %cd PySDM
        !git pull
    !pip install -r requirements.txt
    !ldconfig

In [2]:
import os, sys
if 'google.colab' in sys.modules:
    %cd /content/PySDM
else:
    sys.path.insert(0, os.path.join(os.getcwd(), '../..'))

In [3]:
from PySDM_examples.Arabas_et_al_2015_Figs_8_9.settings import Settings
from PySDM_examples.Arabas_et_al_2015_Figs_8_9.simulation import Simulation
from PySDM_examples.Arabas_et_al_2015_Figs_8_9.storage import Storage
from PySDM_examples.Arabas_et_al_2015_Figs_8_9.netcdf_exporter import NetCDFExporter
from PySDM_examples.Arabas_et_al_2015_Figs_8_9.demo_plots import _TimeseriesPlot, _ImagePlot

from PySDM_examples.utils.temporary_file import TemporaryFile
from PySDM_examples.utils.show_plot import show_plot
from PySDM_examples.utils.widgets import display, HTML, FloatProgress

from PySDM.products.state import AerosolSpecificConcentration, ParticleMeanRadius, DrizzleConcentration

from PySDM.physics import si

import numpy as np
from scipy.io.netcdf import netcdf_file
from matplotlib import pyplot, rcParams
from matplotlib.animation import FuncAnimation
from matplotlib.gridspec import GridSpec

In [4]:
class ProgBarController:
    def __init__(self):
        self.progress = FloatProgress(value=0.0, min=0.0, max=1.0)
        self.panic = False

    def __enter__(self):
        self.set_percent(0)
        display(self.progress)
        
    def __exit__(self, *_):
        pass
    
    def set_percent(self, value):
        self.progress.value = value

In [5]:
settings = Settings()

products = [
    AerosolSpecificConcentration(radius_threshold=settings.aerosol_radius_threshold),
    ParticleMeanRadius(),
    DrizzleConcentration(radius_threshold=settings.drizzle_radius_threshold)
]

settings.n_sd_per_gridbox = 32
settings.grid = (64, 64)
settings.dt = 16 * si.second
settings.n_spin_up = int(si.hour / settings.dt)
settings.n_steps = int(1.75 * settings.n_spin_up)
settings.outfreq = int(1 * si.minute / settings.dt)
settings.condensation_rtol_x = 1e-3
settings.condensation_rtol_thd = 1e-3
settings.adaptive = True

storage = Storage()
simulation = Simulation(settings, storage)
simulation.reinit(products)

In [6]:
simulation.run(ProgBarController())

FloatProgress(value=0.0, max=1.0)

In [7]:
temp_file = TemporaryFile('.nc')
exporter = NetCDFExporter(storage, settings, simulation, temp_file.absolute_path)
exporter.run(ProgBarController())

FloatProgress(value=0.0, max=1.0)

In [14]:
default_figsize = rcParams["figure.figsize"]
figsize = (2.2 * default_figsize[0], 1.3* default_figsize[1])
gs = GridSpec(nrows=9, ncols=27)

fig = pyplot.figure(figsize=figsize)
axs = (
    fig.add_subplot(gs[:gs.nrows-4, 1:gs.ncols//3-1]),
    fig.add_subplot(gs[:gs.nrows-4, gs.ncols//3+1:2*gs.ncols//3-1]),
    fig.add_subplot(gs[:gs.nrows-4, 2*gs.ncols//3+1:-1]),
    fig.add_subplot(gs[gs.nrows-3:-1,2:-1])
)
gs.tight_layout(fig)
ncdf = netcdf_file(temp_file.absolute_path, mode='r', mmap=False)

# TODO: we should not use products here - all info should be obtained from netCDF
plots = (
    _ImagePlot(fig, axs[0], grid=ncdf.grid, size=ncdf.size, product=products[0], cmap='summer'),
    _ImagePlot(fig, axs[1], grid=ncdf.grid, size=ncdf.size, product=products[1], cmap='PuBuGn'),
    _ImagePlot(fig, axs[2], grid=ncdf.grid, size=ncdf.size, product=products[2], cmap='Blues'),
    _TimeseriesPlot(fig, axs[3], ncdf.variables['T'][:], show=False)
)

plots[3].ax.axvline(ncdf.n_spin_up * ncdf.dt)
plots[3].ax.set_ylim(0, .001)

interval = 100 #ms
frame_list = np.arange(ncdf.variables['T'].shape[0], dtype=np.int)

def update(frame_num):
    step = frame_num*ncdf.outfreq
    plots[0].update(ncdf.variables['n_a_mg'][frame_num], step=step)
    plots[1].update(ncdf.variables['radius_m1'][frame_num], step=step)
    plots[2].update(ncdf.variables['n_d_cm3'][frame_num], step=step)
    precip = np.full_like(ncdf.variables['surf_precip'][:], np.nan)
    precip[0:frame_num+1] = ncdf.variables['surf_precip'][0:frame_num+1]
    plots[3].update(precip)
    return (plots[0].im, plots[1].im, plots[2].im, plots[3].timeseries)
pyplot.close(fig)

In [15]:
animation = FuncAnimation(fig, update, frames=frame_list, interval=interval, blit=False)

if 'CI' not in os.environ:
    display(HTML(animation.to_html5_video()))
    file = TemporaryFile('.gif')
    animation.save(file.absolute_path)
    display(file.make_link_widget())

HTML(value='<video width="950" height="374" controls autoplay loop>\n  <source type="video/mp4" src="data:vide…

HTML(value="<a href='../utils/temporary_files/tmpr5xeyg_m.gif' target='_blank'>../utils/temporary_files/tmpr5x…

In [18]:
# save last frame in vector format
svg_file = TemporaryFile('.svg')
fig.savefig(svg_file.absolute_path)
display(svg_file.make_link_widget())

HTML(value="<a href='../utils/temporary_files/tmp4sfe1sog.svg' target='_blank'>../utils/temporary_files/tmp4sf…